# Семинар

In [ ]:
!pip install --upgrade gensim

%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors

In [4]:
import gensim
gensim.__version__

'3.8.3'

In [ ]:
# загрузка модели
from gensim.models import Word2Vec, KeyedVectors

!wget -c https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
!tar -xzf araneum_none_fasttextcbow_300_5_2018.tgz

model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)

In [16]:
#проверка наличия слова в словаре

lemma = 'заграница'
lemma in model

True

## Получение вектора документа

Отлично, вектора для слов получены. Что с ними делать дальше? 

Есть два подхода (а точнее есть один, а второй мы придумали, потому что с одним жить нельзя).
> Классика - для получения вектора документа нужно взять и усреднить все вектора его слов
 
$$ vec_{doc} = \frac {\sum_{i=0}^{n} vec_i}{len(d)} $$

In [ ]:
import numpy as np

# сделали препроцессинг, получили леммы 
lemmas = ['старинный', 'замок']

# создаем вектор-маску
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
print(lemmas_vectors.shape)
vec = np.zeros((model.vector_size,))

# если слово есть в модели, берем его вектор
for idx, lemma in enumerate(lemmas):
    if lemma in model:
        lemmas_vectors[idx] = model[lemma]
        
# проверка на случай, если на вход пришел пустой массив
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)

(2, 300)


> Эксперимент - представим документ не в виде одного уредненного вектора, а как матрицу векторов входящих в него слов

```
 слово1 |  v1_300
 слово2 |  v2_300
 слово3 |  v3_300
 слово4 |  v4_300
```

> Отлично, теперь каждый документ представлен в виде матрицы векторов своих слов. Но нам надо получить близость матрицы документа в коллекции и матрицы входящего запроса. Как? Умножим две матрицы друг на друга - одна матрица размером d x 300, другая q x 300 - получим попарную близость слов из каждого документа - матрицу размером d x q.


In [ ]:
# возьмем игрушечный пример кейса

text1 = 'турция' 
text2 = 'нужна справка срочно'
query = 'быстрая справка'

In [ ]:
# построим матрицы всех документов

def normalize_vec(v):
     return v / np.sqrt(np.sum(v ** 2))

def create_doc_matrix(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        if lemma in model.wv:
            lemmas_vectors[idx] = normalize_vec(model.wv[lemma])
            
    return lemmas_vectors    


text1_m = create_doc_matrix(text1)
text2_m = create_doc_matrix(text2)
query_m = create_doc_matrix(query)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# размер матрицы как и ожидали
query_m.shape

(2, 300)

In [ ]:
# посмотрим на близость слов первого текста и слов запроса
text1_m.dot(query_m.T)

array([[0.09587915, 0.01183069]])

In [ ]:
# посмотрим на близость слов второго текста и слов запроса
text2_m.dot(query_m.T)

array([[-0.0260624 ,  0.11607588],
       [ 0.01341236,  1.00000011],
       [ 0.22505549,  0.33582122]])

In [ ]:
docs_m = [text1_m, text2_m]

def search(docs, query, reduce_func=np.max, axis=0):
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    print(sims)
    return np.argmax(sims)


search(docs_m, query_m)

[0.10770983955697251, 1.225055597288777]


1

# Реализуйте поиск по нашему стандартному Covid корпусу с помощью модели на Araneum двумя способами:

    1. преобразуйте каждый документ в вектор через усреднение векторов его слов и реализуйте поисковик как 
    обычно через умножение матрицы документов коллекции на вектор запроса 
    2. экспериментальный способ - реализуйте поиск ближайшего документа в коллекции к запросу, преобразовав 
    каждый документ в матрицу (количество слов x размер модели)
    
Посчитайте качество поиска для каждой модели на тех же данных, что и в предыдущем задании. В качестве препроцессинга используйте две версии - с удалением NER и без удаления.

In [8]:
import pandas as pd

# это таблички, сохраненные после предыдущих домашек, там уже есть тексты:
# 1. Тексты вопросов без NER DEEPMINT
# 2. Тексты вопросов без NER NATASHA
# + они же после препросессинга

answers = pd.read_excel('/content/answers.xlsx')
queries = pd.read_excel('/content/queries.xlsx')

In [ ]:
answers.head(1)

,Unnamed: 0,Номер связки,Текст вопросов,Текст ответов,Тематика,Текст вопросов без NER DEEPMINT,Текст вопросов без NER NATASHA,deepmint_preprocessed,natasha_preprocessed
0,0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы. Могу ли я...,ребёнок школа продлить каникулы мочь взять бо...,ребёнок школа продлить каникулы мочь взять бо...


In [ ]:
queries.head(1)

,Unnamed: 0,Текст вопросов,Номер связки,Тематика,Текст вопросов без NER NATASHA,Текст вопросов без NER DEEPMINT,deepmint_preprocessed,natasha_preprocessed
0,0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",с уважением \n\nПосле 15 августа 2020 года к н...,с уважением \n\nПосле 15 августа 2020 года ...,уважение 15 август 2020 год мы планировать пр...,уважение 15 август 2020 год мы планировать пр...


In [ ]:
!pip install razdel
!pip install pymorphy2

import pandas as pd
import numpy as np
from razdel import tokenize
from razdel import sentenize
import nltk
from nltk.corpus import stopwords
from string import punctuation
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
nltk.download('stopwords')  
nltk.download('punkt')
russian_stopwords = stopwords.words('russian')

def preprocessing(text: str) -> str:
  tokens = list(tokenize(text))
  tokens = [_.text for _ in tokens]
  tokens = [word.lower() for word in tokens if word.lower() not in russian_stopwords]
  tokens = [(token.translate(str.maketrans('', '', punctuation))) for token in tokens]
  lemmas = [morph.parse(token)[0].normal_form for token in tokens]

  return " ".join(lemmas)

In [10]:
preprocessed_texts_plain = [preprocessing(str(text)) for text in list(answers['Текст вопросов'])]
answers['preprocessed_texts_plain'] = preprocessed_texts_plain
answers.head(1)

,Unnamed: 0,Номер связки,Текст вопросов,Текст ответов,Тематика,Текст вопросов без NER DEEPMINT,Текст вопросов без NER NATASHA,deepmint_preprocessed,natasha_preprocessed,preprocessed_texts_plain
0,0,57,У ребенка в школе продлили каникулы. Могу ли я...,Листок временной нетрудоспособности (больничны...,БОЛЬНИЧНЫЙ ЛИСТ,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы. Могу ли я...,ребёнок школа продлить каникулы мочь взять бо...,ребёнок школа продлить каникулы мочь взять бо...,ребёнок школа продлить каникулы мочь взять бо...


In [11]:
preprocessed_texts_plain = [preprocessing(str(text)) for text in list(queries['Текст вопросов'])]
queries['preprocessed_texts_plain'] = preprocessed_texts_plain
queries.head(1)

,Unnamed: 0,Текст вопросов,Номер связки,Тематика,Текст вопросов без NER NATASHA,Текст вопросов без NER DEEPMINT,deepmint_preprocessed,natasha_preprocessed,preprocessed_texts_plain
0,0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",с уважением \n\nПосле 15 августа 2020 года к н...,с уважением \n\nПосле 15 августа 2020 года ...,уважение 15 август 2020 год мы планировать пр...,уважение 15 август 2020 год мы планировать пр...,уважение вероника игоревич ильич 15 август 202...


# МАТРИЦЫ

In [12]:
from sklearn.model_selection import train_test_split

columns = ['Номер связки', 'preprocessed_texts_plain', 'deepmint_preprocessed', 'natasha_preprocessed']

answers_train = pd.DataFrame(answers, columns=columns)
queries_train = pd.DataFrame(queries, columns=columns)

train_quer, test = train_test_split(queries_train, test_size=0.3, random_state=7654)
train = pd.concat([answers_train, train_quer])
print(train.shape, test.shape)

(1652, 4) (690, 4)


In [13]:
# построим матрицы всех документов

def normalize_vec(v):
     return v / np.sqrt(np.sum(v ** 2))

def create_doc_matrix(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        if lemma in model.wv:
            lemmas_vectors[idx] = normalize_vec(model.wv[lemma])
            
    return lemmas_vectors    

def search(docs, query, reduce_func=np.max, axis=0):
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    return np.argmax(sims)

In [ ]:
plain_matrix = [create_doc_matrix(str(text)) for text in train['preprocessed_texts_plain']]
natasha_matrix = [create_doc_matrix(str(text)) for text in train['natasha_preprocessed']]
deepmint_matrix = [create_doc_matrix(str(text)) for text in train['deepmint_preprocessed']]

In [ ]:
from tqdm import tqdm

links = list(train['Номер связки'])

plain_matrix_test = [create_doc_matrix(str(text)) for text in tqdm(test['preprocessed_texts_plain'])]
plain_score = [links[search(plain_matrix , text)] for text in tqdm(plain_matrix_test)]

In [ ]:
natasha_matrix_test = [create_doc_matrix(str(text)) for text in tqdm(test['natasha_preprocessed'])]
natasha_score = [links[search(natasha_matrix , text)] for text in tqdm(natasha_matrix_test)]

In [ ]:
deepmint_matrix_test = [create_doc_matrix(str(text)) for text in tqdm(test['deepmint_preprocessed'])]
deepmint_score = [links[search(deepmint_matrix , text)] for text in tqdm(deepmint_matrix_test)]

In [ ]:
from sklearn.metrics import accuracy_score

test = test.fillna(0)
test.astype({'Номер связки': 'int32'}).dtypes

In [30]:
test['plain_score'] = plain_score
test = test.fillna(0)
test.astype({'plain_score': 'int32'}).dtypes

accuracy_score(test['Номер связки'], test['plain_score'])

0.47101449275362317

In [32]:
test['natasha_score'] = natasha_score
test = test.fillna(0)
test.astype({'natasha_score': 'int32'}).dtypes

accuracy_score(test['Номер связки'], test['natasha_score'])

0.4623188405797101

In [34]:
test['deepmint_score'] = deepmint_score
test = test.fillna(0)
test.astype({'deepmint_score': 'int32'}).dtypes

accuracy_score(test['Номер связки'], test['deepmint_score'])

0.4608695652173913

# ВЕКТОРИЦАЗИЯ

In [49]:
def normalize_vec(v):
     return v / np.sqrt(np.sum(v ** 2))

def make_vec(text):
  # сделали препроцессинг, получили леммы 
  lemmas = text.split()

  # создаем вектор-маску
  lemmas_vectors = np.zeros((len(lemmas), model.vector_size))

  vec = np.zeros((model.vector_size,))

  # если слово есть в модели, берем его вектор
  for idx, lemma in enumerate(lemmas):
      if lemma in model:
          lemmas_vectors[idx] = model[lemma]
          
  # проверка на случай, если на вход пришел пустой массив
  if lemmas_vectors.shape[0] is not 0:
      vec = np.mean(lemmas_vectors, axis=0)

  return vec

In [48]:
from sklearn.model_selection import train_test_split

columns = ['Номер связки', 'preprocessed_texts_plain', 'deepmint_preprocessed', 'natasha_preprocessed']

answers_train = pd.DataFrame(answers, columns=columns)
queries_train = pd.DataFrame(queries, columns=columns)

train_quer, test = train_test_split(queries_train, test_size=0.3, random_state=7654)
train = pd.concat([answers_train, train_quer])
print(train.shape, test.shape)

(1652, 4) (690, 4)


In [51]:
plain_vec = [normalize_vec(make_vec(str(text))) for text in train['preprocessed_texts_plain']]
natasha_vec = [normalize_vec(make_vec(str(text))) for text in train['natasha_preprocessed']]
deepmint_vec = [normalize_vec(make_vec(str(text))) for text in train['deepmint_preprocessed']]

In [52]:
plain_martix = np.array(plain_vec)
natasha_martix = np.array(natasha_vec)
deepmint_martix = np.array(deepmint_vec)

In [53]:
def sim_vectors(matrix, query):
  doc = normalize_vec(make_vec(query))
  sim = matrix.dot(doc.T)
  return sim

In [ ]:
plain_matrix_test = [sim_vectors(plain_martix, str(text)) for text in test['preprocessed_texts_plain']]
plain_score = [links[np.argmax(text)] for text in tqdm(plain_matrix_test)]

natasha_matrix_test = [sim_vectors(natasha_martix, str(text)) for text in test['natasha_preprocessed']]
natasha_score = [links[np.argmax(text)] for text in tqdm(natasha_matrix_test)]

deepmint_matrix_test = [sim_vectors(deepmint_martix, str(text)) for text in test['deepmint_preprocessed']]
deepmint_score = [links[np.argmax(text)] for text in tqdm(deepmint_matrix_test)]

In [58]:
test['plain_score_vec'] = plain_score
test = test.fillna(0)
test.astype({'plain_score_vec': 'int32'}).dtypes

accuracy_score(test['Номер связки'], test['plain_score_vec'])

0.5289855072463768

In [59]:
test['natasha_score_vec'] = natasha_score
test = test.fillna(0)
test.astype({'natasha_score_vec': 'int32'}).dtypes

accuracy_score(test['Номер связки'], test['natasha_score_vec'])

0.5057971014492754

In [61]:
test['deepmint_score_vec'] = deepmint_score
test = test.fillna(0)
test.astype({'deepmint_score_vec': 'int32'}).dtypes

accuracy_score(test['Номер связки'], test['deepmint_score_vec'])

0.49130434782608695